In [1]:
!pip install pygame numpy metakernel

  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.1.0
    Uninstalling traitlets-5.1.0:
      Successfully uninstalled traitlets-5.1.0
  Attempting uninstall: pywin32
    Found existing installation: pywin32 228
    Uninstalling pywin32-228:
      Successfully uninstalled pywin32-228
  Attempting uninstall: jupyter-core
    Found existing installation: jupyter-core 4.8.1
    Uninstalling jupyter-core-4.8.1:
      Successfully uninstalled jupyter-core-4.8.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


In [2]:
!python -m pygame.examples.aliens

pygame 2.3.0 (SDL 2.24.2, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import pygame
import numpy as np
import time
import math
from IPython.display import HTML
import random
import threading
import types
import traceback
#from toggle_input import hide_toggle as hide
from collections import defaultdict 
def def_value(): 
    return None
stateManager = defaultdict(def_value) 

class GameThread(threading.Thread):
    def __init__(self,  *args, **kwargs):
        super(GameThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    
#Initial Config
class Game:
    def __init__(self,display_width,display_height,window_color,title):
        self.size = (display_width,display_height)
        self.title = title
        self.running = True
        self.scene = list()
        self.window_color=window_color
        self.event_handlers = dict()
        self.event_handlers[(('type',pygame.QUIT),)] = self.on_quit
        self.event_handlers[(('type',pygame.KEYDOWN), ('key',pygame.K_q))] = self.on_quit
        self.event_handlers[(('type',pygame.KEYDOWN), ('key',pygame.K_ESCAPE))] = self.on_quit
        self.flipdelay=16
        self.tickcounter=0
    ##############################################################################################    
    def render(self):
        self.disp.fill((0,0,0))
        for obj in self.scene:
            try:
                obj.render(self.disp)
            except Exception:
                traceback.print_exc()
                self.scene.remove(obj)
                print("Rendering Error: Object "+str(obj)+" removed from the scene")
        try:
            tile_group.draw(self.disp)
            all_sprites.draw(self.disp)
        except Exception:
            traceback.print_exc()
        # unlike .update() just updates the screen as a whole          
        pygame.display.flip()
    def update(self):
        dt=pygame.time.get_ticks()- self.tickcounter
        for obj in self.scene:
            try:
                obj.update(dt)
            except Exception:
                traceback.print_exc()
                self.scene.remove(obj)
                print("Update Error: Object "+str(obj)+" removed from the scene")
        all_sprites.update()
        tile_group.update()
        self.tickcounter=pygame.time.get_ticks()
        pygame.time.delay(self.flipdelay)
    
    def on_quit(self, event):
        self.running = False
    
    def process_events(self):
        for event in pygame.event.get():
            dire = dir(event)
            for eh in self.event_handlers.keys():
                callit=True
                for (attrname,attrvalue) in eh:
                    if (not attrname in dire) or (event.__getattribute__(attrname)!=attrvalue):
                        callit=False
                        break
                if callit:
                    self.event_handlers[eh](event)
        
    def display(self):
        pygame.init()
        self.disp=pygame.display.set_mode(self.size, pygame.HWSURFACE | pygame.DOUBLEBUF)
        pygame.display.set_caption(self.title)
        self.clock=pygame.time.Clock()
        self.tickcounter=pygame.time.get_ticks()
        while( self.running ):
            try:
                self.render()
                self.process_events()
                self.update()
            except Exception:
                traceback.print_exc()
                pygame.time.delay(10000)
        pygame.quit()

def startGame(display_width=500,display_height=500,window_color=(0,0,0),title='NoteG Prototype'):
    global game,engine,scene,WIDTH,FPS,HEIGHT
    WIDTH = display_width
    HEIGHT = display_height
    FPS = 60
    game = Game(display_width,display_height,window_color,title)
    game.refresh_scene = types.MethodType(refresh_scene, game)
    engine = GameThread(target = game.display)
    engine.start()

def getSize():
    return pygame.display.get_surface().get_size()
    

In [5]:
all_sprites = pygame.sprite.Group()
player_sprites = pygame.sprite.Group()
player_bullets = pygame.sprite.Group()
enemy_bullets = pygame.sprite.Group()
enemy_sprites = pygame.sprite.Group()
tile_group = pygame.sprite.Group()
wall_group = pygame.sprite.Group()
tile_map  = [[4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4],
            [5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5],
            [5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5],
            [5,1,1,1,3,2,2,2,2,2,2,2,2,2,5,1,1,3,2,2,2,2,2,2,2,2,2,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,5,1,1,3,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5,1,1,1,5],
            [5,1,1,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,1,1,1,5],
            [5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5],
            [5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5],
            [5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,5],
            [5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,1,1,4,4,4,4,4,4,4,4,4,4,4,4,4,5],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0],
            [5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0],]
#Primary Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
DARK_GREEN = (34, 177, 76)

BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
RED = (255, 0, 0)

LEGO_BLUE = (0,50,150)
LEGO_ORANGE = (255,150,40)

VIOLET = (181, 126, 220)
ORANGE = (255, 165, 0)
GREEN = (0, 128, 0)
GRAY = (128, 128, 128)


#Extended Colors
IVORY = (255, 255, 240)
BEIGE = (245, 245, 220)
WHEAT = (245, 222, 179)
TAN = (210, 180, 140)
KHAKI = (195, 176, 145)
SILVER = (192, 192, 192)
CHARCOAL = (70, 70, 70)
NAVYBLUE = (0, 0, 128)
ROYALBLUE = (8, 76, 158)
MEDIUMBLUE = (0, 0, 205)
AZURE = (0, 127, 255)
CYAN = (0, 255, 255)
AQUAMARINE = (127, 255, 212)
TEAL = (0, 128, 128)
FORESTGREEN = (34, 139, 34)
OLIVE = (128, 128, 0)
LIME = (191, 255, 0)
GOLD = (255, 215, 0)
SALMON = (250, 128, 114)
HOTPINK = (252, 15, 192)
FUCHSIA = (255, 119, 255)
PUCE = (204, 136, 153)
PLUM = (132, 49, 121)
INDIGO = (75, 0, 130)
MAROON = (128, 0, 0)
CRIMSON = (220, 20, 60)
DEFAULT = (0, 0, 0)
colorlist = [
            BLACK,
            WHITE,
            BLUE,
            YELLOW,
            RED,
            VIOLET,
            ORANGE,
            GREEN,
            GRAY,
            IVORY,
            BEIGE,
            WHEAT,
            TAN,
            KHAKI,
            SILVER,
            CHARCOAL,
            NAVYBLUE,
            ROYALBLUE,
            MEDIUMBLUE,
            AZURE,
            CYAN,
            AQUAMARINE,
            TEAL,
            FORESTGREEN,
            OLIVE,
            LIME,
            GOLD,
            SALMON,
            HOTPINK,
            FUCHSIA,
            PUCE,
            PLUM,
            INDIGO,
            MAROON,
            CRIMSON,
            DEFAULT
            ]

In [6]:
class SpriteSheet(object):
    """ Class used to grab images out of a sprite sheet. """
    # This points to our sprite sheet image
    sprite_sheet = None

    def __init__(self, file_name):
        """ Constructor. Pass in the file name of the sprite sheet. """

        # Load the sprite sheet.
        self.sprite_sheet = pygame.image.load(file_name).convert()


    def get_image(self, x, y, width, height):
        """ Grab a single image out of a larger spritesheet
            Pass in the x, y location of the sprite
            and the width and height of the sprite. """

        # Create a new blank image
        image = pygame.Surface([width, height]).convert()
        # Copy the sprite from the large sheet onto the smaller image
        image.blit(self.sprite_sheet, (0, 0), (x, y, width, height))

        # Return the image
        return image

In [7]:
# for simple texture
class Tiles:
    def __init__(self,tile):
        self.tile = pygame.image.load(tile).convert()
        
    def update(self,dt):
        return
    ##############################################################################################
    def render(self, disp):
        for y in range(0,500,32):
            for x in range(0,500,32):
                disp.blit(self.tile, (x,y))

# for complex tile maps
class Tile(pygame.sprite.Sprite):
    def __init__(self, image, is_solid):
        pygame.sprite.Sprite.__init__(self)
        self.image = image
        self.rect = self.image.get_rect()
        self.solid_state = is_solid

    def update(self):
        self.rect.x += -player.speedx
        self.rect.y += -player.speedy
    ##############################################################################################
def createMap(lvl,grass,floor_one,wall_bottom,wall_right,wall_top,wall_left):
    for y, row in enumerate(lvl):
        for x, terrain_num in enumerate(row):
            if terrain_num == 0:
                None

            elif terrain_num == 1:
                tile = Tile(floor_one, False)
                tile.rect.center = (x * 64, y * 64)
                tile_group.add(tile)

            elif terrain_num == 2:
                tile = Tile(wall_bottom, True)
                tile.rect.center = (x * 64, y * 64)
                tile_group.add(tile)
                wall_group.add(tile)

            elif terrain_num == 3:
                tile = Tile(wall_right, True)
                tile.rect.center = (x * 64, y * 64)
                tile_group.add(tile)
                wall_group.add(tile)

            elif terrain_num == 4:
                tile = Tile(wall_top, True)
                tile.rect.center = (x * 64, y * 64)
                tile_group.add(tile)
                wall_group.add(tile)

            elif terrain_num == 5:
                tile = Tile(wall_left, True)
                tile.rect.center = (x * 64, y * 64)
                tile_group.add(tile)
                wall_group.add(tile)

In [8]:
class Bullet(pygame.sprite.Sprite):
    def __init__(self, trigger, x, y):
        pygame.sprite.Sprite.__init__(self)
        
        if trigger.ai:
            self.image = pygame.transform.scale(fairydust, (60, 20))
        else:
            self.image = pygame.transform.scale(arrow_img, (60, 20))
        
        self.image.set_colorkey(DARK_GREEN)
        self.rect = self.image.get_rect()
        self.rect.bottom = y
        self.rect.centerx = x
        self.speedx = 0
        self.speedy = 0
        self.flipx = False
        self.trigger = trigger
        if self.trigger.ai:
            if self.trigger.direction == "right":
                self.speedx = -5
                self.image = pygame.transform.rotate(self.image, 0)

            if self.trigger.direction == "left":
                self.speedx = 5
                self.image = pygame.transform.rotate(self.image, 180)

            if self.trigger.direction == "down":
                self.speedy = -5
                self.image = pygame.transform.rotate(self.image, -90)

            if self.trigger.direction == "up":
                self.speedy = 5
                self.image = pygame.transform.rotate(self.image, 90)
        else:
            if self.trigger.direction == "left":
                self.speedx = -5
                self.image = pygame.transform.rotate(self.image, 0)

            if self.trigger.direction == "right":
                self.speedx = 5
                self.image = pygame.transform.rotate(self.image, 180)

            if self.trigger.direction == "up":
                self.speedy = -5
                self.image = pygame.transform.rotate(self.image, -90)

            if self.trigger.direction == "down":
                self.speedy = 5
                self.image = pygame.transform.rotate(self.image, 90)

    def update(self):
        self.scrolling()
        self.rect.y += self.speedy
        self.rect.x += self.speedx
        # kill if it moves off the top of the screen
        if self.rect.bottom < 0 or self.rect.top> HEIGHT:
            self.kill()
        if self.rect.left < 0 or self.rect.right > WIDTH:
            self.kill()

    def scrolling(self):
        self.rect.x += -player.speedx
        self.rect.y += -player.speedy

In [10]:
class Character(pygame.sprite.Sprite):
    def __init__(self,char, x, y, ai=False):
        pygame.sprite.Sprite.__init__(self)
        sprite_sheet = SpriteSheet(char)
        self.walking_frames_d = [] # Images for animated walking.
        self.walking_frames_u = []
        self.walking_frames_l = []
        self.walking_frames_r = []
        self.direction= ""
        self.stat = dict()
        for i in range(8):
            image = sprite_sheet.get_image(64+i*64,0,64,64)
            image.set_colorkey(DARK_GREEN)
            self.walking_frames_u.append(image)
        image = sprite_sheet.get_image(0,0,64,64)
        image.set_colorkey(DARK_GREEN)
        self.stat["up"] = image

        for i in range(8):
            image = sprite_sheet.get_image(64+i*64,128,64,64)
            image.set_colorkey(DARK_GREEN)
            self.walking_frames_d.append(image)
        image = sprite_sheet.get_image(0,128,64,64)
        image.set_colorkey(DARK_GREEN)
        self.stat["down"] = image

        for i in range(8):
            image = sprite_sheet.get_image(64+i*64,64,64,64)
            image.set_colorkey(DARK_GREEN)
            self.walking_frames_l.append(image)
        image = sprite_sheet.get_image(0,64,64,64)
        image.set_colorkey(DARK_GREEN)
        self.stat["left"] = image

        for i in range(8):
            image = sprite_sheet.get_image(64+i*64,192,64,64)
            image.set_colorkey(DARK_GREEN)
            self.walking_frames_r.append(image)
        image = sprite_sheet.get_image(0,192,64,64)
        image.set_colorkey(DARK_GREEN)
        self.stat["right"] = image
        
        self.image = self.walking_frames_d[0]
        self.rect = self.image.get_rect()
        self.rect.center = (x-32, y-32)
        self.speedx = 0
        self.speedy = 0
        
        self.frames = self.walking_frames_d
        
        self.coeff = 2
        self.last_update = pygame.time.get_ticks()
        self.frame = 0

        self.shoot_delay = 250
        self.last_shot = pygame.time.get_ticks()
        self.ai = ai
        if ai:
            self.speedx = -1
            self.speedy = -1
            self.shoot_delay = 500

    def update(self):
        if not self.ai:
            self.speedx = 0
            self.speedy = 0
            self.coeff = 1
            self.check_bullets()
            keystate = pygame.key.get_pressed()
            if(any(keystate)):
                collision_types = self.check_wall()
                if keystate[pygame.K_UP]:
                    self.speedy = -2*self.coeff*(-20 if collision_types['top'] else 1)
                    self.animate_sprite(self.walking_frames_u)
                    self.direction = "up"
                if keystate[pygame.K_DOWN]:
                    self.speedy = 2*self.coeff*(-20 if collision_types['bottom'] else 1)
                    self.animate_sprite(self.walking_frames_d)
                    self.direction = "down"
                if keystate[pygame.K_LEFT]:
                    self.speedx = -2*self.coeff*(-20 if collision_types['left'] else 1)
                    self.animate_sprite(self.walking_frames_l)
                    self.direction = "left"
                if keystate[pygame.K_RIGHT]:
                    self.speedx = 2*self.coeff*(-20 if collision_types['right'] else 1)
                    self.animate_sprite(self.walking_frames_r)
                    self.direction = "right"

                if keystate[pygame.K_RIGHT] and keystate[pygame.K_UP]:
                    self.direction = "up"

                if keystate[pygame.K_LEFT] and keystate[pygame.K_UP]:
                    self.direction = "up"

                if keystate[pygame.K_DOWN] and keystate[pygame.K_RIGHT]:
                    self.direction = "down"

                if keystate[pygame.K_DOWN] and keystate[pygame.K_LEFT]:
                    self.direction = "down"

                if keystate[pygame.K_SPACE]:
                    self.shoot()
        else:
            self.animate_sprite(self.frames)
            self.check_bullets()
            self.scrolling()
            self.move_towards_player(player)
            self.check_direction() 
            if self.rect.x >= player.rect.x - 20 or self.rect.x <= player.rect.x + 20:
                self.shoot()

            if self.rect.y >= player.rect.y - 20 or self.rect.y <= player.rect.y + 20:
                self.shoot()
        now = pygame.time.get_ticks()
        if self.speedx==0 and self.speedy == 0 and self.direction!="":
            self.last_update = now
            self.frame = 0
            self.image = self.stat[self.direction]
    def animate_sprite(self, anim_list):
        now = pygame.time.get_ticks()
        if now - self.last_update > 100:
            self.last_update = now
            self.frame = (self.frame + 1) % 7
            self.image = anim_list[self.frame]

    def shoot(self):
        now = pygame.time.get_ticks()
        if now - self.last_shot > self.shoot_delay:
            self.last_shot = now
            bullet = Bullet(self,self.rect.centerx, self.rect.centery + 35)
            all_sprites.add(bullet)
            if not self.ai:
                player_bullets.add(bullet)
            if self.ai:
                enemy_bullets.add(bullet)
                
    def scrolling(self):
        self.rect.x += -player.speedx
        self.rect.y += -player.speedy
    def check_direction(self):
        if abs(self.dx) > abs(self.dy):
            if self.dx >= 0:
                self.frames = self.walking_frames_l
                self.direction = "right"
            elif self.dx < 0:
                self.frames = self.walking_frames_r
                self.direction = "left"
        else:
            if self.dy >= 0:
                self.frames = self.walking_frames_u
                self.direction = "down"
            elif self.dy < 0:
                self.frames = self.walking_frames_d
                self.direction = "up"
    def move_towards_player(self, player):
        self.dx = self.rect.x - player.rect.x
        self.dy = self.rect.y - player.rect.y
        dist = math.hypot(self.dx, self.dy)
        try:
            self.dx = self.dx /dist
            self.dy = self.dy / dist
        except ZeroDivisionError:
            dist = 1
        self.rect.x += self.dx * self.speedx
        self.rect.y += self.dy * self.speedy  
    def check_bullets(self):
        enemy_collisions = pygame.sprite.groupcollide(wall_group, player_bullets, False, True)
    def check_wall(self):
        collision_types = {'top':False,'bottom':False,'right':False,'left':False}
        wall_collisions = pygame.sprite.spritecollideany(player, wall_group)
    
#         enemy_collisions = pygame.sprite.groupcollide(enemy_sprites, player_bullets, False, False)
#         print(tile_group)
#         for collisions in wall_collisions:
#             print(collision)
        if wall_collisions:
            if wall_collisions.solid_state:
                if(player.rect.left<=wall_collisions.rect.right):
                    collision_types['left'] = True
                if(player.rect.top<=wall_collisions.rect.bottom):
                    collision_types['top'] = True
                if(player.rect.bottom>=wall_collisions.rect.top):
                    collision_types['bottom'] = True
                if(player.rect.right>=wall_collisions.rect.left):
                    collision_types['right'] = True
        
        return collision_types

In [11]:
class Trinket(pygame.sprite.Sprite):
    def __init__(self,shape="circle",speed=4, image=None, center=None, **kwargs):
        pygame.sprite.Sprite.__init__(self)
        if type(center) in [list,tuple]:
            self.x = center[0]
            self.y = center[1]
        else:
            self.x = random.randint(0,WIDTH)
            self.y = random.randint(-40,-20)
        self.dirx=1
        self.img=None
        if image!=None:
            self.img=pygame.image.load(image)
            self.rect = self.img.get_rect()
        else:
            self.rect = kwargs.get('rect', (self.x,self.y,20,20))
        self.shape=shape
        self.speed=speed
        self.kwargs = kwargs
    ##############################################################################################
    def render(self, display):
        if not self.img:
            if self.shape == "circle":
                pygame.draw.circle(display, **self.kwargs, center=(self.x, self.y))
            else:
                # not checked for all shapes
                func = getattr(pygame.draw, shape)
                func(self.kwargs)
        else:
            display.blit(self.img,self.rect)        
    
    def update(self,dt):
#         self.check_bullets()
        if self.x>500:
            self.dirx=-1
        if self.x<0:
            self.dirx=1
        self.x+=self.dirx*self.speed

In [12]:
def refresh_scene(self,tile):
    self.scene=list()
    self.player = Character(char_sheet,100,100)
    self.scene.append(Tiles(tile))
#     self.scene.append(self.player)
#     self.event_handlers[(('type',pygame.KEYDOWN),)] = self.player.handle_keydown
Game.refresh_scene = refresh_scene

In [13]:
def repeatMap(tile):
    game.scene.append(Tiles(tile))
def createPlayer(name,char_sheet,x=100,y=100,ai=False):
    global player
    player = Character(char_sheet,x,y)
    game.player = player
    stateManager[name] = len(game.scene)
#     game.scene.append(player)
    all_sprites.add(player)
def createEnemy(char_sheet,x=100,y=100,ai=True):
    global enemy
    enemy = Character(char_sheet,x,y,ai)
#     stateManager[name] = len(game.scene)
#     game.scene.append(player)
    enemy_sprites.add(enemy)
    all_sprites.add(enemy)

In [14]:
def stopBullets():
    for bullet in player_bullets:
        bullet.speedx = 0
        bullet.speedy = 0

In [15]:
def changeElement(key):
    global element
    element = key